<a href="https://colab.research.google.com/github/Jahid1990/Chatbot/blob/main/Step%20by%20step%20Chatbot%20building%20in%20telegram%20using%20python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBoT

In this notebook, I will try to demostrate end to end  building process of telegram chatbot based in python. Chatbot has 3 parts if I breakdown components of a basic chatbot. 

1. Input
2. Understanding the Input
3. Provide answer based on  understanding the Input

I will use telegram as interface- to take input and show output, deep learning LSTM model to understand the Intent of the input.

lets explore......

# Required python libraries

In [ ]:
# !pip install bnlp_toolkit
import numpy as np # linear algebra
import pandas as pd 
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
pd.set_option("display.max_rows", 50000, "display.max_columns", 500)

# Training Data loading

I have around 88k queries and corresponding answers of that queries. I used this dataset for building this chatbot. Questions and answers would be limited to this dataset.*italicized text* 

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/to_english.csv")
data=data.iloc[:,1:3]
data.columns=["Text","Intent"]
data.head()

,Text,Intent
0,Amar account e koto taka ache?,checkavailablebalanceand validity
1,Ami account balance kivabe dekhbo?,checkavailablebalanceand validity
2,Amar koto taka balance ase?,checkavailablebalanceand validity
3,Amar koto taka ache?,checkavailablebalanceand validity
4,Account er balance bolen,checkavailablebalanceand validity


# Text Processsing

Did not know that keras provide such wonderful functions for text processing. There are some few other libraries in pyhton which explored some of them but to me keras is awesome for its simplicity. I am happy using keras for text processing as well as training our intent understanding model.

following cells could be titled as follows:

1. Tokenizing, Sequencing and Padding
2. Train Test Splitting
3. Model Architecture
4. Compliling the mode along with loss function definition
5. Training model 

In [ ]:
max_fatures =2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ',char_level=True)
tokenizer.fit_on_texts(data['Text'].values)
#tokenizer.index_word=dict(pd.Series(tokenizer.index_word.values())[~pd.Series(tokenizer.index_word.values()).isin(stop_words)])
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X,padding='post')

In [ ]:
Y = pd.get_dummies(data['Intent'],prefix_sep='a')
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(59385, 102) (59385, 33)
(29250, 102) (29250, 33)


In [ ]:
embed_dim = 196
lstm_out = 200

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, return_sequences=True))
# model.add(LSTM(lstm_out, dropout=0.2, return_sequences=True))
# model.add(LSTM(lstm_out, dropout=0.2))
model.add(tf.keras.layers.Flatten())
model.add(Dense(Y_train.shape[1],activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 102, 196)          392000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 102, 196)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 102, 200)          317600    
_________________________________________________________________
flatten_2 (Flatten)          (None, 20400)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 33)                673233    
Total params: 1,382,833
Trainable params: 1,382,833
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import tensorflow as tf
learn_rate = 0.0001
optim = tf.keras.optimizers.Adam(lr=learn_rate, decay=1e-5)
checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/gp_chatbot.h5', verbose=1, save_best_only=True)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

In [ ]:
batch_size = 512
learn_rate = 0.0001
optim = tf.keras.optimizers.Adam(lr=learn_rate, decay=1e-5)
model.fit(X_train, Y_train, epochs = 100, batch_size=batch_size,validation_split=.2, verbose = 1)

Epoch 1/100
93/93 [==============================] - 7s 78ms/step - loss: 0.0206 - accuracy: 0.9932 - val_loss: 0.0823 - val_accuracy: 0.9779
Epoch 2/100
93/93 [==============================] - 5s 57ms/step - loss: 0.0198 - accuracy: 0.9931 - val_loss: 0.0772 - val_accuracy: 0.9798
Epoch 3/100
93/93 [==============================] - 5s 57ms/step - loss: 0.0204 - accuracy: 0.9930 - val_loss: 0.0771 - val_accuracy: 0.9784
Epoch 4/100
93/93 [==============================] - 5s 57ms/step - loss: 0.0196 - accuracy: 0.9934 - val_loss: 0.0762 - val_accuracy: 0.9789
Epoch 5/100
93/93 [==============================] - 5s 56ms/step - loss: 0.0192 - accuracy: 0.9936 - val_loss: 0.0790 - val_accuracy: 0.9788
Epoch 6/100
93/93 [==============================] - 5s 57ms/step - loss: 0.0186 - accuracy: 0.9937 - val_loss: 0.0766 - val_accuracy: 0.9796
Epoch 7/100
93/93 [==============================] - 5s 57ms/step - loss: 0.0184 - accuracy: 0.9940 - val_loss: 0.0773 - val_accuracy: 0.9783
Epoch 

#Evaluation

In [ ]:
from sklearn.metrics import classification_report
import tensorflow as tf
model=tf.keras.models.load_model('/content/drive/MyDrive/gp_chatbot.h5')
pred=model.predict(X_test)                                                                                                                                                                          
pred=np.argmax(pred,axis=1)
act=np.argmax(Y_test.to_numpy(),axis=1)
print(classification_report(pred, act))

In [ ]:
data["PRED"]=Y_test.columns[pd.DataFrame(pred).idxmax(axis=1)]
data["PRED_ACC"]=np.where(data.Intent==data.PRED,1,0)
data["PRED_ACC"].value_counts()/data.shape[0]*100
data

,Text,Intent,PRED,PRED_ACC
0,Amar account e koto taka ache?,checkavailablebalanceand validity,CheckSMSofferinfo,0
1,Ami account balance kivabe dekhbo?,checkavailablebalanceand validity,CheckSMSofferinfo,0
2,Amar koto taka balance ase?,checkavailablebalanceand validity,CheckSMSofferinfo,0
3,Amar koto taka ache?,checkavailablebalanceand validity,CheckSMSofferinfo,0
4,Account er balance bolen,checkavailablebalanceand validity,CheckSMSofferinfo,0
...,...,...,...,...
88630,What special minit offer there in my number,MicroVoiceOfferInfo,CheckSMSofferinfo,0
88631,Any minit offer are in my number,MicroVoiceOfferInfo,CheckSMSofferinfo,0
88632,Inform me about my minit offer in this number,MicroVoiceOfferInfo,CheckSMSofferinfo,0
88633,Which menit offer are available in my number,MicroVoiceOfferInfo,CheckSMSofferinfo,0


#Create a Basic BOT in telegram

1. Type "BotFather" in telegram search bar. You will find a bot named "BotFather". Type "/newbot"  and it will give following response "Alright, a new bot. How are we going to call it? Please choose a name for your bot."
2. After giving a name of your bot it will give following message "Good. Now let's choose a username for your bot. It must end in `bot`. Like this, for example: TetrisBot or tetris_bot."
So, following above guidelines give a username of your bot. example of user name could be: GPdemoChatBot.  
3. Done. Note token provided after step 2 when required in the upcoming steps. 

---



# Finally Run the following cell and enjoy



In [ ]:
#!pip install pyTelegramBotAPI
#libraries
import telebot
import tensorflow as tf
import numpy as np 
import pandas as pd 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# model load
model=tf.keras.models.load_model('/content/drive/MyDrive/gp_chatbot.h5')# plz customize path for your case 
# tokenizing
data=pd.read_csv("/content/drive/MyDrive/to_english.csv")
data=data.iloc[:,1:3]
data.columns=["Text","Intent"]
max_fatures =2000

API_TOKEN = '1525093601:AAGeK8859-JyAjn_jMu_0Fwu97WCq2foEQs'

bot = telebot.TeleBot(API_TOKEN)


# Handle '/start' and '/help'
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    bot.reply_to(message, """\
Hi there, I am GP ChatBoT.
I am here to answer your query!\
""")


# Handle all other messages with content_type 'text' (content_types defaults to ['text'])
@bot.message_handler(func=lambda message: True)
def echo_message(message):
    if pd.Series(message.text).isin(['HI','Hi','hi','hI','hello','Hello','HELLO']).bool(): 
      bot.reply_to(message,'Hi there ! I am here to answer your queries. Please let me know how can I help you')
    else:
      tokenizer = Tokenizer(num_words=max_fatures, split=' ')
      tokenizer.fit_on_texts(data.Text)
      columns=pd.get_dummies(data.Intent,prefix_sep='a').columns

      
      twt = tokenizer.texts_to_sequences(message.text)
    
      twt = pad_sequences(twt,padding='post',maxlen=23,value=0.0,dtype='int32')
      sentiment = model.predict(twt)
      bot.reply_to(message, columns[pd.DataFrame(sentiment).idxmax(axis=1)][0])

bot.polling()
while True:pass

# Convert all text to English

In [ ]:
#!pip install google_trans_new

from google_trans_new import google_translator  
translator = google_translator()  
trans=[]
l=0
for i in data.iloc[:,0]:
  trans.append(translator.translate(i,lang_tgt='en')) 
  l=l+1 
  print(l)